# Entrenamiento de los modelos

## Librerías utilizadas

In [3]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA, TruncatedSVD, FactorAnalysis
from sklearn.random_projection import johnson_lindenstrauss_min_dim
from sklearn.random_projection import SparseRandomProjection

## Lectura del dataframe

In [4]:
dfPath = "dataframe/train_df.csv"
train_df = pd.read_csv(dfPath)

In [10]:
# Verificar documentos con solo una oración (sentence_num = 0 únicamente)
# Contar oraciones por ID
oraciones_por_id = train_df.groupby('id')['sentence_num'].apply(lambda x: x.tolist())

# Filtrar IDs que solo tienen sentence_num = 0
ids_una_oracion = oraciones_por_id[oraciones_por_id.apply(lambda x: x == [0])].index

print(f"Número de documentos con solo una oración: {len(ids_una_oracion)}")
print(f"Total de documentos: {train_df['id'].nunique()}")
print(f"Porcentaje: {len(ids_una_oracion) / train_df['id'].nunique() * 100:.2f}%")

# Mostrar df con los documentos con solo una oración
df_una_oracion = train_df[train_df['id'].isin(ids_una_oracion)]
display(df_una_oracion)

Número de documentos con solo una oración: 36
Total de documentos: 1000
Porcentaje: 3.60%


,id,sentence_num,model,domain,POS_VERB,POS_NOUN,POS_ADJ,POS_ADV,POS_DET,POS_INTJ,...,RE,ASF,ASM,OM,RCI,DMC,OR,QAS,PA,PR
615,49,0,human,reddit,0.279762,0.130952,0.071429,0.047619,0.083333,0.000000,...,0.000000,0.000000,0.053571,0.000000,0.0,0.000000,0.000000,0.059524,0.0,0.000000
688,59,0,mpt,poetry,0.294430,0.233422,0.005305,0.005305,0.172414,0.000000,...,0.000000,0.000000,0.000000,0.122016,0.0,0.000000,0.000000,0.172414,0.0,0.000000
976,87,0,human,poetry,0.239216,0.133333,0.078431,0.050980,0.074510,0.007843,...,0.000000,0.000000,0.019608,0.011765,0.0,0.007843,0.000000,0.031373,0.0,0.000000
1156,107,0,human,poetry,0.174757,0.184466,0.072816,0.087379,0.077670,0.004854,...,0.000000,0.004854,0.019417,0.019417,0.0,0.000000,0.000000,0.063107,0.0,0.000000
1412,126,0,gpt2,poetry,0.000000,0.500000,0.000000,0.000000,0.231481,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
1413,127,0,mpt-chat,poetry,0.156463,0.231293,0.040816,0.068027,0.115646,0.006803,...,0.000000,0.000000,0.013605,0.000000,0.0,0.000000,0.000000,0.061224,0.0,0.000000
2682,232,0,mpt,books,0.249453,0.260394,0.124726,0.111597,0.021882,0.008753,...,0.002188,0.002188,0.017505,0.004376,0.0,0.008753,0.002188,0.013129,0.0,0.002188
3252,285,0,human,reddit,0.265000,0.035000,0.060000,0.060000,0.025000,0.030000,...,0.000000,0.000000,0.035000,0.005000,0.0,0.000000,0.000000,0.025000,0.0,0.000000
3563,314,0,llama-chat,poetry,0.166667,0.303922,0.117647,0.039216,0.117647,0.000000,...,0.000000,0.009804,0.029412,0.000000,0.0,0.000000,0.000000,0.078431,0.0,0.000000
3714,326,0,human,reddit,0.000000,0.008130,0.000000,0.000000,0.000000,0.016260,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000


In [4]:
print("Información del dataset de entrenamiento:")
print(f"Forma del dataset: {train_df.shape}")
print(f"Columnas: {list(train_df.columns)}")
print(f"Modelos unicos: {train_df['model'].unique()}")
print(f"Dominios unicos: {train_df['domain'].unique()}")

Información del dataset de entrenamiento:
Forma del dataset: (12168, 200)
Columnas: ['id', 'sentence_num', 'model', 'domain', 'POS_VERB', 'POS_NOUN', 'POS_ADJ', 'POS_ADV', 'POS_DET', 'POS_INTJ', 'POS_CONJ', 'POS_PART', 'POS_NUM', 'POS_PREP', 'POS_PRO', 'L_REF', 'L_HASHTAG', 'L_MENTION', 'L_RT', 'L_LINKS', 'L_CONT_A', 'L_FUNC_A', 'L_CONT_T', 'L_FUNC_T', 'L_PLURAL_NOUNS', 'L_SINGULAR_NOUNS', 'L_PROPER_NAME', 'L_PERSONAL_NAME', 'L_NOUN_PHRASES', 'L_PUNCT', 'L_PUNCT_DOT', 'L_PUNCT_COM', 'L_PUNCT_SEMC', 'L_PUNCT_COL', 'L_PUNCT_DASH', 'L_POSSESSIVES', 'L_ADJ_POSITIVE', 'L_ADJ_COMPARATIVE', 'L_ADJ_SUPERLATIVE', 'L_ADV_POSITIVE', 'L_ADV_COMPARATIVE', 'L_ADV_SUPERLATIVE', 'PS_CONTRADICTION', 'PS_AGREEMENT', 'PS_EXAMPLES', 'PS_CONSEQUENCE', 'PS_CAUSE', 'PS_LOCATION', 'PS_TIME', 'PS_CONDITION', 'PS_MANNER', 'SY_QUESTION', 'SY_NARRATIVE', 'SY_NEGATIVE_QUESTIONS', 'SY_SPECIAL_QUESTIONS', 'SY_TAG_QUESTIONS', 'SY_GENERAL_QUESTIONS', 'SY_EXCLAMATION', 'SY_IMPERATIVE', 'SY_SUBORD_SENT', 'SY_SUBORD_SENT

## Preparación de datos

In [5]:
# 1. Crear etiqueta binaria: 1 = Humano (model == 'human'), 0 = IA (resto)
print("Distribución de clases:")
print((train_df['model'] == 'human').value_counts())
print(f"\nTotal de oraciones: {len(train_df)}")

Distribución de clases:
model
True     6839
False    5329
Name: count, dtype: int64

Total de oraciones: 12168


In [6]:
# 2. Definir columnas de features (excluir metadatos y target)
# Excluir: id, sentence_num, model, domain
metadata_cols = ['id', 'sentence_num', 'model', 'domain']
feature_columns = [col for col in train_df.columns if col not in metadata_cols]

print(f"\nTotal de features: {len(feature_columns)}")
print(f"Primeros 10 features: {feature_columns[:10]}")


Total de features: 196
Primeros 10 features: ['POS_VERB', 'POS_NOUN', 'POS_ADJ', 'POS_ADV', 'POS_DET', 'POS_INTJ', 'POS_CONJ', 'POS_PART', 'POS_NUM', 'POS_PREP']


In [7]:
# 3. Obtener IDs únicos y crear mapping de ID a clase
unique_ids = train_df['id'].unique()
print(f"\nTotal de documentos únicos: {len(unique_ids)}")

# Mapping de ID a clase (binaria: humano vs IA)
id_to_class = train_df.groupby('id')['model'].first().to_dict()


Total de documentos únicos: 1000


In [8]:
# Verificar cuántos documentos hay por clase
print("Distribución de documentos por clase:")
class_distribution = pd.Series(id_to_class.values()).value_counts()
print(class_distribution)
print(f"\nClase 0 (IA): {class_distribution.get(0, 0)} documentos")
print(f"Clase 1 (Humano): {class_distribution.get(1, 0)} documentos")

Distribución de documentos por clase:
human           500
gpt2             64
mistral-chat     62
mpt-chat         61
mpt              58
mistral          54
llama-chat       50
chatgpt          35
gpt3             34
cohere           31
cohere-chat      27
gpt4             24
Name: count, dtype: int64

Clase 0 (IA): 500 documentos
Clase 1 (Humano): 64 documentos


C:\Users\HP\AppData\Local\Temp\ipykernel_23852\198586682.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"\nClase 0 (IA): {class_distribution.get(0, 0)} documentos")
C:\Users\HP\AppData\Local\Temp\ipykernel_23852\198586682.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Clase 1 (Humano): {class_distribution.get(1, 0)} documentos")


In [9]:
# 4. Split de IDs (no de oraciones) - IMPORTANTE para evitar data leakage
ids_list = list(id_to_class.keys())
labels_list = [id_to_class[id_] for id_ in ids_list]

# Verificar si podemos usar stratify
class_counts = pd.Series(labels_list).value_counts()
can_stratify = class_counts.min() >= 2

if can_stratify:
    print("✓ Usando stratified split (mantiene proporción de clases)")
    train_ids, test_ids = train_test_split(
        ids_list, 
        test_size=0.2, 
        random_state=42, 
        stratify=labels_list
    )
else:
    print(f"⚠️ No se puede usar stratify (clase mínima: {class_counts.min()} documentos)")
    print("Usando split aleatorio sin stratify")
    train_ids, test_ids = train_test_split(
        ids_list, 
        test_size=0.2, 
        random_state=42, 
        stratify=None  # Sin estratificación
    )

print(f"\nDocumentos en train: {len(train_ids)}")
print(f"Documentos en test: {len(test_ids)}")

✓ Usando stratified split (mantiene proporción de clases)

Documentos en train: 800
Documentos en test: 200


In [10]:
# 5. Filtrar oraciones según los IDs
train_sentences = train_df[train_df['id'].isin(train_ids)]
test_sentences = train_df[train_df['id'].isin(test_ids)]

print(f"\nOraciones en train: {len(train_sentences)}")
print(f"Oraciones en test: {len(test_sentences)}")

# Verificar distribución en cada conjunto
print(f"\nDistribución en train:")
# print(train_sentences['is_human'].value_counts())
print(train_sentences['model'].value_counts())
print(f"\nDistribución en test:")
# print(test_sentences['is_human'].value_counts())
print(test_sentences['model'].value_counts())


Oraciones en train: 9594
Oraciones en test: 2574

Distribución en train:
model
human           5395
gpt2             745
llama-chat       491
mistral          467
mistral-chat     466
mpt              441
chatgpt          382
mpt-chat         290
cohere           275
gpt4             251
gpt3             209
cohere-chat      182
Name: count, dtype: int64

Distribución en test:
model
human           1444
gpt2             270
mpt              129
chatgpt          123
mistral-chat     117
llama-chat       116
mistral           96
mpt-chat          74
gpt4              66
gpt3              62
cohere            45
cohere-chat       32
Name: count, dtype: int64


## Preparación de features

In [11]:
# 6. Preparar X e y
X_train = train_sentences[feature_columns].values
y_train = train_sentences['model'].apply(lambda x: 1 if x == 'human' else 0).values
X_test = test_sentences[feature_columns].values
y_test = test_sentences['model'].apply(lambda x: 1 if x == 'human' else 0).values

print(f"\n{'='*60}")
print(f"Shape de X_train: {X_train.shape}")
print(f"Shape de y_train: {y_train.shape}")
print(f"Shape de X_test: {X_test.shape}")
print(f"Shape de y_test: {y_test.shape}")
print(f"{'='*60}")


Shape de X_train: (9594, 196)
Shape de y_train: (9594,)
Shape de X_test: (2574, 196)
Shape de y_test: (2574,)


In [12]:
# Verificar si hay valores NaN en los datos
print("Verificando valores NaN en el dataset...")
print(f"\nNaNs en train_df: {train_df.isna().sum().sum()}")
print(f"NaNs en feature_columns:")
nan_features = train_df[feature_columns].isna().sum()
nan_features_with_nans = nan_features[nan_features > 0]
if len(nan_features_with_nans) > 0:
    print(nan_features_with_nans)
else:
    print("No hay NaNs en las features ✓")

print(f"\nNaNs en X_train: {np.isnan(X_train).sum()}")
print(f"NaNs en X_test: {np.isnan(X_test).sum()}")
print(f"NaNs en y_train: {np.isnan(y_train).sum()}")
print(f"NaNs en y_test: {np.isnan(y_test).sum()}")

Verificando valores NaN en el dataset...

NaNs en train_df: 39
NaNs en feature_columns:
ST_HERDAN_TTR    39
dtype: int64

NaNs en X_train: 31
NaNs en X_test: 8
NaNs en y_train: 0
NaNs en y_test: 0


In [13]:
# Solución: Imputar valores NaN antes de entrenar
from sklearn.impute import SimpleImputer

# Opción 1: Imputar con la media de cada feature
imputer = SimpleImputer(strategy='mean')

# Ajustar el imputer con los datos de entrenamiento
X_train_clean = imputer.fit_transform(X_train)
# Transformar los datos de test con el mismo imputer
X_test_clean = imputer.transform(X_test)

print("✓ Valores NaN imputados con la media de cada feature")
print(f"NaNs en X_train_clean: {np.isnan(X_train_clean).sum()}")
print(f"NaNs en X_test_clean: {np.isnan(X_test_clean).sum()}")

✓ Valores NaN imputados con la media de cada feature
NaNs en X_train_clean: 0
NaNs en X_test_clean: 0


## Reducción de dimensionalidad

### PCA

In [14]:
pca = PCA().fit(X_train_clean)

explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)
num_components = np.argmax(cumulative_variance >= 0.95) + 1

print(f"Número de componentes para explicar el 95% de la varianza: {num_components}")

Número de componentes para explicar el 95% de la varianza: 24


In [15]:
pca = PCA(n_components=24)
X_train_pca = pca.fit_transform(X_train_clean)
X_test_pca = pca.transform(X_test_clean)

### LDA

In [16]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# Aplicar LDA para reducir la dimensionalidad
lda = LinearDiscriminantAnalysis(n_components=1)  # La cantidad de componentes debe ser <= número de clases - 1
x_train_lda = lda.fit_transform(X_train_clean, y_train)
x_test_lda = lda.transform(X_test_clean)

### FA

In [17]:
fa = FactorAnalysis().fit(X_train_clean)

singular_values = fa.components_
explained_variance = np.var(singular_values, axis=1) / np.var(X_train_clean, axis=0).sum()

# Calcula la varianza explicada acumulativa
cumulative_explained_variance = np.cumsum(explained_variance)

# Determina el número de factores necesarios para explicar al menos el 95% de la varianza
num_factors = np.argmax(cumulative_explained_variance >= 0.95) + 1

print(f"Number of factors to retain: {num_factors}")

Number of factors to retain: 1


In [18]:
fa = FactorAnalysis(n_components=1)
x_train_fa = fa.fit_transform(X_train_clean)
x_test_fa = fa.transform(X_test_clean)

### SVD

In [19]:
svd = TruncatedSVD(n_components=min(X_train_clean.shape) - 1)
svd.fit(X_train_clean)

# Calcula la varianza explicada por los componentes
explained_variance = svd.explained_variance_ratio_

# Calcula la varianza explicada acumulativa
cumulative_explained_variance = np.cumsum(explained_variance)

# Determina el número de componentes necesarios para explicar al menos el 95% de la varianza
num_components = np.argmax(cumulative_explained_variance >= 0.95) + 1

print(f"Number of components to retain: {num_components}")

Number of components to retain: 25


In [20]:
svd = TruncatedSVD(n_components=25)
x_train_svd = svd.fit_transform(X_train_clean)
x_test_svd = svd.transform(X_test_clean)

### JL

In [21]:
epsilon = 0.9

# Calcula el número mínimo de componentes necesarios usando la Proyección de Johnson-Lindenstrauss
n_samples = X_train_clean.shape[0]
n_components = johnson_lindenstrauss_min_dim(n_samples, eps=epsilon)
print(f"Number of components to retain using JL: {n_components}")

Number of components to retain using JL: 226


In [22]:
jl = SparseRandomProjection(n_components=224)
x_train_jl = jl.fit_transform(X_train_clean)
x_test_jl = jl.transform(X_test_clean)

c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\random_projection.py:411: DataDimensionalityWarning: The number of components is higher than the number of features: n_features < n_components (196 < 224).The dimensionality of the problem will not be reduced.
  warnings.warn(


## Entrenamiento del modelo

### Función de resultados

In [23]:
def showResults(y_test, y_pred):
    print(f"\n{'='*60}")
    print("RESULTADOS EN TEST SET (Split por Documento)")
    print(f"{'='*60}")
    print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.4f}")

    print("\n--- Classification Report ---")
    print(classification_report(y_test, y_pred, target_names=['IA', 'Humano'], digits=4))

    print("\n--- Confusion Matrix ---")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print(f"\nInterpretación:")
    print(f"  TN (IA correctamente clasificada): {cm[0,0]}")
    print(f"  FP (IA clasificada como Humano): {cm[0,1]}")
    print(f"  FN (Humano clasificado como IA): {cm[1,0]}")
    print(f"  TP (Humano correctamente clasificado): {cm[1,1]}")

### SVM

In [24]:
trainX, testX = X_train_clean, X_test_clean
# trainX, testX = X_train_pca, X_test_pca
# trainX, testX = x_train_lda, x_test_lda
# trainX, testX = x_train_fa, x_test_fa
# trainX, testX = x_train_svd, x_test_svd
# trainX, testX = x_train_jl, x_test_jl

C = 10

print("="*60)

svm_model = SVC(kernel='rbf', C=C, gamma='scale', random_state=42)
svm_model.fit(trainX, y_train)

y_pred = svm_model.predict(testX)

print("✓ Modelo entrenado exitosamente")

✓ Modelo entrenado exitosamente


### Mostrar resultados

In [25]:
showResults(y_test, y_pred)


RESULTADOS EN TEST SET (Split por Documento)

Accuracy: 0.6418

--- Classification Report ---
              precision    recall  f1-score   support

          IA     0.6368    0.4283    0.5122      1130
      Humano     0.6439    0.8089    0.7170      1444

    accuracy                         0.6418      2574
   macro avg     0.6404    0.6186    0.6146      2574
weighted avg     0.6408    0.6418    0.6271      2574


--- Confusion Matrix ---
[[ 484  646]
 [ 276 1168]]

Interpretación:
  TN (IA correctamente clasificada): 484
  FP (IA clasificada como Humano): 646
  FN (Humano clasificado como IA): 276
  TP (Humano correctamente clasificado): 1168
